In [1]:
from azureml.core import Workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: crypto-prediction-project
Azure region: eastus2
Resource group: crypto_prediction_resource


In [2]:
ws.get_details()

{'applicationInsights': '/subscriptions/509b3593-2de3-40ce-a6b8-a838635aecb6/resourcegroups/crypto_prediction_resource/providers/microsoft.insights/components/cryptopredicti1485413605',
 'containerRegistry': '/subscriptions/509b3593-2de3-40ce-a6b8-a838635aecb6/resourceGroups/crypto_prediction_resource/providers/Microsoft.ContainerRegistry/registries/cryptopredice150b3df',
 'creationTime': '2019-08-04T01:35:47.4531218+00:00',
 'description': '',
 'friendlyName': '',
 'id': '/subscriptions/509b3593-2de3-40ce-a6b8-a838635aecb6/resourceGroups/crypto_prediction_resource/providers/Microsoft.MachineLearningServices/workspaces/crypto-prediction-project',
 'identityPrincipalId': '82f68c0b-24dc-4c4e-a0d2-093d4bc05789',
 'identityTenantId': '47bdc275-03cb-4246-8049-32f26432f195',
 'identityType': 'SystemAssigned',
 'keyVault': '/subscriptions/509b3593-2de3-40ce-a6b8-a838635aecb6/resourcegroups/crypto_prediction_resource/providers/microsoft.keyvault/vaults/cryptopredicti0282800410',
 'location': '

In [3]:
from azureml.core.model import Model
# model = Model.register(model_path = "price_predictor.pt",
#                        model_name = "price_predictor.pt",
#                        tags = {'type': "regression"},
#                        description = "BTC Price prediction with PyTorch",
#                        workspace = ws
#                       )

model = Model(name='price_predictor.pt',workspace=ws)
print('Registered model = ', model)

Registered model =  Model(workspace=Workspace.create(name='crypto-prediction-project', subscription_id='509b3593-2de3-40ce-a6b8-a838635aecb6', resource_group='crypto_prediction_resource'), name=price_predictor.pt, id=price_predictor.pt:1, version=1, tags={'type': 'regression'}, properties={})


In [4]:
!ls

model_deployment.ipynb	price_predictor.pt  scoring_file_testing.ipynb
myenv.yml		score.py


In [5]:
model_path = Model.get_model_path(model_name='price_predictor.pt')

In [6]:
model_path

'price_predictor.pt'

In [82]:
%%writefile score.py
import json
import time
import sys
import os
from azureml.core.model import Model
import numpy as np    # we're going to use numpy to process input and output data
import torch
import torch.nn as nn
import torch.nn.functional as F
import requests
import pandas as pd
from sklearn import preprocessing

class TimeRNN(nn.Module):
    def __init__(self,bat_size,in_features,h_size,layer_amnt):
        super(TimeRNN,self).__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.batch_sz = bat_size
        self.in_features = in_features
        self.h_size = h_size
        self.layer_amnt = layer_amnt
        
        self.lstm1 = nn.LSTM(input_size=self.in_features,
                             hidden_size=self.h_size,
                             num_layers=self.layer_amnt,
                             bias=True,
                             batch_first=False,
                             dropout=0,
                             bidirectional=False)
        self.fc1 = nn.Linear(in_features=1,out_features=1)
    def init_hidden(self):
        """Intialize/re-init the hidden and cell states. 
        The hidden state acts as the memory of the RNN 
        which gets passed from one unit to another. 
        h_i = f(h_i + in)

        Intializing with 0s
        """
        #print('layer size =\t', self.layer_amnt)
        #print('bat_size =\t', self.batch_sz)
        #print('hidden size =\t',self.h_size)
        return (torch.zeros(self.layer_amnt,self.batch_sz,self.h_size),
                torch.zeros(self.layer_amnt,self.batch_sz,self.h_size))
    def forward(self,x):
        x = x.unsqueeze(0)
        hidden_init = self.init_hidden()
        h0 = hidden_init[0].to(self.device)
        c0 = hidden_init[1].to(self.device)
        x,hidden = self.lstm1( x,(h0,c0))
        x = F.leaky_relu(self.fc1(x[-1].view(self.batch_sz,-1)))
        return x

class Inferencer(object):
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    def open_model(self,location):
        model = TimeRNN(bat_size=1,in_features=3,h_size=1,layer_amnt=1)
        model.load_state_dict(torch.load(location))
        model.eval()
        return model

    def un_normalize(self,norm_val,min_val,max_val,typelist=None):
        if(typelist):
            for idx,item in enumerate(norm_val):
                new_val = item * (max_val - min_val) + min_val
                norm_val[idx] = new_val
            return norm_val
        else:
            return norm_val * (max_val - min_val) + min_val 

    def inference(self,value, normalize_method, model,minimum_price,maximum_price):
        value = np.array(value)
        predictions = []
        for sample in value:
            sample = np.array(sample).reshape(1,-1)
            example = torch.tensor(normalize_method.transform(sample)).float()
            
            if(str(self.device) == 'cuda'):
                example = example.to(self.device)

            output = model(example)
            output_unnorm = self.un_normalize(norm_val=output.detach(),min_val=minimum_price,max_val=maximum_price)
            predictions.append(np.array(output_unnorm).item())
        return predictions

    def fetch_latest_BTC_JSON(self):
        """Fetch the latest JSON data"""
        API_LINK = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey=SAITMI5ZUMGEKGKY'
        page = requests.get(API_LINK).json()
        return page
    def parse_alphaV_JSON(self,raw_data):
        # Remove meta data for now
        raw_data.pop('Meta Data',None)
        # Remove key name
        df = pd.DataFrame.from_dict(raw_data['Time Series (Digital Currency Daily)'],dtype=float)
        # Flip dates as columns into rows
        df = df.transpose()
        return df


def init():
    global model
    inf = Inferencer()
    model_path = Model.get_model_path(model_name='price_predictor.pt')
    #print(model_path)
    #model = torch.load(model_path, map_location=lambda storage, loc: storage)
    model = inf.open_model(location=model_path)
    #model.eval()
    
def preprocess(input_data):
    """JSON Input conversion into the tensor input"""
    input_data = json.loads(input_data)['data']
    # -- Normalize --
    inf = Inferencer()
    raw_data = inf.fetch_latest_BTC_JSON()
    df = inf.parse_alphaV_JSON(raw_data=raw_data)
    prices = np.array(df['4a. close (USD)'].tolist())
    data_df_temp = df.drop(labels=['1a. open (USD)','1b. open (USD)','2b. high (USD)','3b. low (USD)','4a. close (USD)','4b. close (USD)','6. market cap (USD)'],axis=1)
    minmax_2 = preprocessing.MinMaxScaler()
    data_df_temp = pd.DataFrame(minmax_2.fit_transform(data_df_temp), columns=data_df_temp.columns)

    minimum_price = np.min(prices)
    maximum_price = np.max(prices)
    return input_data,minmax_2,minimum_price,maximum_price
    

def postprocess(result):
    
    result = np.array(result).item()
    
    inf = Inferencer()
    
    raw_data = inf.fetch_latest_BTC_JSON()
    df = inf.parse_alphaV_JSON(raw_data=raw_data)
    prices = np.array(df['4a. close (USD)'].tolist())

    minimum_price = np.min(prices)
    maximum_price = np.max(prices)

    res = inf.un_normalize(norm_val=result,min_val=minimum_price,max_val=maximum_price)
    return res

def run(input_data_json):
    try:
        start = time.time()   # start timer
        input_data,normalizer,min_price,max_price = preprocess(input_data_json) 

        #output = model(input_data)
        #res = postprocess(output.detach())
        inf = Inferencer()
        res = inf.inference(value=input_data,
                            normalize_method=normalizer,
                            model=model,
                            minimum_price=min_price,
                            maximum_price=max_price
                           )
        end = time.time()     # stop timer
        return {"result": res,"time": end - start}
    except Exception as e:
        result = str(e)
        return {"error": result}

Overwriting score.py


In [83]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.set_python_version('3.5.2')
myenv.add_pip_package("scikit-learn")
myenv.add_pip_package("numpy==1.15.1")
myenv.add_pip_package("torch==1.1.0")
myenv.add_pip_package("matplotlib==2.2.3")
myenv.add_pip_package("pyyaml==3.11")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
print(myenv.serialize_to_string())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.5.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

  - scikit-learn
  - numpy==1.15.1
  - torch==1.1.0
  - matplotlib==2.2.3
  - pyyaml==3.11
channels:
- conda-forge



In [84]:
type(model)

azureml.core.model.Model

## Create Image Container 

In [85]:
from azureml.core.image import Image, ContainerImage
image_config = ContainerImage.image_configuration(runtime="python",
                                 execution_script="score.py",
                                 conda_file="myenv.yml",
                                 tags = {'type': "regression"},
                                 description = "BTC Price prediction with PyTorch")
image = Image.create(name = "myimage1",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation(show_output = True)

Creating image
Running.................................................
Succeeded
Image creation operation finished for image myimage1:8, operation "Succeeded"


## Create Azure Container Instance WebService

In [87]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                          memory_gb = 1, 
                                          tags = {"type": "regression"}, 
                                          description = 'price prediction')

In [88]:
from azureml.core.webservice import Webservice
service_name = 'aci-price-prediction'
service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                            image = image,
                                            name = service_name,
                                            workspace = ws)
service.wait_for_deployment(show_output = True)

Creating service
Running...............................
SucceededACI service creation operation finished, operation "Succeeded"


In [86]:
service.delete()

In [89]:
import json
# test_sample = json.dumps({'data': [
#     [[8700,11080,35000]], 
# ]})


test_sample = json.dumps({'data': [
    [8700,11080,35000],
    [10800,11090,25000]
]})
test_sample = bytes(test_sample,encoding ='utf8')
print(test_sample)
#print(type(test_sample))
t = service.run(input_data=test_sample)
print(t)

b'{"data": [[8700, 11080, 35000], [10800, 11090, 25000]]}'
{'result': [8616.0498046875, 10726.2314453125], 'time': 0.9305286407470703}
